In [1]:
import os 
import dotenv 

Define the model and embedding to be used 

In [7]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama2"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [8]:
# test the model
model.invoke("briefly, what's AI?")

'\nArtificial intelligence (AI) is a branch of computer science that focuses on creating intelligent machines capable of performing tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI systems use algorithms and statistical models to analyze data, learn from it, and make decisions or take actions based on that analysis.\n\nThere are several types of AI, including:\n\n1. Narrow or weak AI: This type of AI is designed to perform a specific task, such as image recognition, speech recognition, or language translation. Examples include Siri, Alexa, and Google Assistant.\n2. General or strong AI: This type of AI is designed to perform any intellectual task that a human can do. It is still in the realm of science fiction, but researchers are working towards developing a general AI that can learn, reason, and make decisions like a human.\n3. Superintelligence: This type of AI is significantly more intelligent than the best huma

In [9]:
# create a parser to convert the model output to a string
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# chain the model with output parser
chain = model | parser
chain.invoke("in short sentences, what's AI?")

'\nArtificial intelligence (AI) is:\n\n1. A machine that can think and learn like a human.\n2. A technology that enables computers to perform tasks that typically require human intelligence, such as understanding natural language or recognizing images.\n3. A system that can process and analyze vast amounts of data quickly and accurately.\n4. A tool that can help automate repetitive tasks and make decisions based on patterns and insights.\n5. A revolutionary technology that is transforming industries and changing the way we live and work.'

Define a prompt template to answer questions based on context

In [10]:
from langchain.prompts import PromptTemplate

template = """ 
Answer the question based on the context given below, 
if you don't know the answer, just say "I don't know".

Context: {context}

question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(
    context = "Here is some context",
    question = "Here is the question"
)

' \nAnswer the question based on the context given below, \nif you don\'t know the answer, just say "I don\'t know".\n\nContext: Here is some context\n\nquestion: Here is the question\n\n'

Create a chain that combines the prompt, model and output parser 

In [11]:
chain = prompt | model | parser
chain.invoke({
    "context": "There are three types of machine learning: supervised, unsupervised, and reinforcement learning.",
    "question": "What are the types of machine learning?"
})

'Sure! Based on the context provided, the types of machine learning are:\n\n1. Supervised learning\n2. Unsupervised learning\n3. Reinforcement learning'

In [12]:
from dataclasses import dataclass

@dataclass
class Document:
    page_content: str
    metadata: dict


Load PDF files and convert them into Document objects

In [13]:
# Load PDF using pypdf into list of documents.
# Loader chunks by page and stores page numbers in metadata.
from langchain_community.document_loaders import PyPDFLoader

directory_path = "dl_pdf_files/"
loaders = []
pages = []
files = os.listdir(directory_path)

# Iterate over each file in the directory
for file in files:
    # Only process PDF files
    if file.endswith(".pdf"):
        file_path = os.path.join(directory_path, file)
        loader = PyPDFLoader(file_path)
        loaders.append(loader)
        # Load and split the PDF into pages
        raw_pages = loader.load_and_split()
        for raw_page in raw_pages:
            # Convert raw page to Document
            document = Document(page_content=raw_page.page_content, metadata=raw_page.metadata)
            pages.append(document)

pages[0]


Document(page_content='Chapter 1  I  Introducing deep learning 4\nWhy you should learn deep learning\nIt’s a powerful tool for the incremental automation of intelligence.\nFrom the beginning of time, humans have been building better and better tools to \nunderstand and control the environment around us. Deep learning is today’s chapter in this story of innovation. \nPerhaps what makes this chapter so compelling is that this field is more of a mental  \ninnovation than a mechanical one . Much like its sister fields in machine learning, deep \nlearning seeks to automate intelligence  bit by bit. In the past few years, it has achieved \nenormous success and progress in this endeavor, exceeding previous records in computer \nvision, speech recognition, machine translation, and many other tasks. \nThis is particularly extraordinary given that deep learning seems to use largely the same \nbrain-inspired algorithm (neural networks) for achieving these accomplishments across a vast number of f

Create a vector store from the loaded documents using the embeddings

In [14]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [16]:
# convert the vectorstore to a retriever
retriever = vectorstore.as_retriever()
retriever.invoke("What is a neural network?")

[Document(metadata={'source': 'dl_pdf_files/chapter4.pdf', 'page': 1}, page_content='Chapter 4  I  Introduction to neural learning 48\nPredict, compare, and learn\nIn chapter 3, you learned about the paradigm “predict, compare, learn, ” and we dove \ndeep into the first step: predict . In the process, you learned a myriad of things, including \nthe major parts of neural networks (nodes and weights), how datasets fit into networks (matching the number of datapoints coming in at one time), and how to use a neural network to make a prediction. \nPerhaps this process begged the question, “How do we set weight values so the network \npredicts accurately?” Answering this question is the main focus of this chapter, as we cover the next two steps of the paradigm: compare  and learn .\nCompare\nComparing gives a measurement of how much a prediction \n“missed” by.\nOnce you’ve made a prediction, the next step is to evaluate how well you did. This may \nseem like a simple concept, but you’ll find

Define a chain to process questions using context retrieval and the model

In [17]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt | model | parser 
)

In [19]:
questions = [
    "What is the purpose of the book?",
    "How many chapters there is?",
    "How long it will take to read the whole book?",
    "How to check if the network is making good predictions?",
    "What's the difference between generalization and overfitting?",
    "What is a deep learning framework?",
    "At the end, give me a brief guide please.",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the book?
Answer:  Based on the given context, the purpose of the book appears to be teaching deep learning from scratch, assuming that the reader has a basic understanding of algebra. The author highlights the importance of finding a personal problem to work on and using deep learning to solve it. The book covers installation of Jupyter Notebook and NumPy, as well as mathematical assumptions and how to measure error in the "compare" step of the paradigm "predict, compare, learn".

Question: How many chapters there is?
Answer: Based on the provided PDF documents, there are 4 chapters in total.

Chapter 1: Introduction to neural learning
Chapter 2: Predict, compare, and learn
Chapter 3: Predict with a neural network
Chapter 4: Introduction to neural prediction: forward propagation

Question: How long it will take to read the whole book?
Answer: I don't know how long it will take to read the whole book. The length of time required to read a book depends o

In [21]:
# Process questions in batch
chain.batch([{ "question": question } for question in questions])

["Based on the given context, the purpose of the book appears to be teaching deep learning from scratch, assuming the reader has a basic understanding of algebra. The author wants the reader to find a personal problem they are interested in and use deep learning to solve it. The book covers installation of Jupyter Notebook and NumPy, and assumes the reader will have a basic understanding of mathematical concepts. The author also mentions that having a problem you're fascinated with that involves using one dataset to predict another is key to becoming an expert in the field.",
 'Based on the provided PDF documents, there are 4 chapters in the book.',
 "I don't know exactly how long it will take to read the whole book based on the provided context, as it depends on various factors such as reading speed, comprehension level, and personal interest in the subject matter. However, based on the length of the book and the amount of information presented, it could take anywhere from a few days 

In [22]:
# Stream the answer to a specific question
for s in chain.stream({"question": "What is the purpose of the book?"}):
    print(s, end="", flush=True)

 Based on the provided context, the purpose of the book appears to be teaching deep learning from scratch, assuming the reader has a basic understanding of algebra. The author emphasizes the importance of finding a personal problem to work on, as it can help drive interest and motivation in the subject. The book covers installation of Jupyter Notebook and NumPy, as well as mathematical assumptions and concepts. Additionally, the author discusses the importance of measuring error and learning from it, using examples from real-life situations. Overall, the purpose of the book seems to be providing a comprehensive introduction to deep learning for beginners.